# 🪙 Camada Bronze - Ingestão e Explosão de Itens via Auto Loader

## 📌 Objetivo

Este pipeline tem como objetivo realizar a **ingestão incremental e automatizada de dados de vendas**, oriundos de arquivos Delta armazenados no Data Lake (**camada Bronze**), aplicando transformações com **explode de arrays**, enriquecimento com dados aninhados e gravação otimizada com particionamento por data. 

*A camada Bronze é responsável por armazenar os dados brutos exatamente como foram recebidos, sem transformações. Essa abordagem permite manter um histórico completo, garantindo rastreabilidade e possibilitando futuros reprocessamentos caso necessário.*

É aplicado o Auto Loader para detecção contínua de arquivos, com escrita resiliente no Delta Lake e **registro condicional da tabela no catálogo**, garantindo um pipeline robusto, reutilizável e econômico em recursos.

---

## 📁 Estrutura de Pastas

| Camada     | Localização                                                                 |
|------------|------------------------------------------------------------------------------|
| Bronze     | `abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas`           |
| Bronze Out | `abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas_exploded`  |

---

## 🔍 Fonte de Dados

- **vendas_lucas** — Arquivos Delta contendo pedidos com múltiplos itens e endereço aninhado (arrays e structs).

---

## 🛠️ Transformações Aplicadas

- 🔁 **Explode de arrays**: transforma múltiplos itens por pedido em múltiplas linhas (`explode(Items)`)
- 🔍 **Extração de nested fields**: extrai campos de structs como `ShippingAddress.City`, `Item.Price`, etc.
- 🏷️ **Padronização de colunas**: com `alias()` para clareza semântica
- 📅 **Particionamento por data**: organiza os arquivos por `OrderDate` para ganho de performance
- ⛓️ **Registro dinâmico no catálogo**: cria a tabela automaticamente, caso ainda não esteja registrada

---

## ⚙️ Técnicas Avançadas Utilizadas

| Técnica                         | Benefício                                                                 |
|----------------------------------|--------------------------------------------------------------------------|
| 🔄 `Auto Loader (cloudFiles)`    | Ingestão contínua com detecção de novos arquivos Delta                   |
| 🎯 `explode()`                   | Permite granularidade por item, essencial para análises por produto      |
| 🧊 `Delta Lake`                  | Tolerância a falhas, versionamento e performance com escrita ACID       |
| 🧭 `checkpointLocation`          | Garante consistência e permite retomada segura em falhas                 |
| 📊 `ZORDER BY`                   | Otimiza performance de leitura com filtros por `OrderID` e `CustomerID` |
| 🔍 `Verificação condicional`     | Cria a tabela no catálogo apenas se ainda não existir                    |

---

## 🧪 Validação Final

Após a escrita, o pipeline executa:

- ✅ **Contagem total de registros gravados**
- 🚀 **Otimização com `OPTIMIZE ZORDER`**
- 📚 **Registro automático da tabela no metastore**

---

## 🔐 Governança e Reusabilidade

- Tabela registrada no metastore (`bronze.vendas_lucas_exploded`)
- Compatível com SQL, notebooks e visualizações
- Código modular, reutilizável e com dependências explícitas

---

## 👨‍💻 Stack Utilizada

- **Databricks** (Notebook)
- **Apache Spark Structured Streaming**
- **Delta Lake**
- **Auto Loader**
- **Azure Data Lake Storage Gen2 (ABFSS)**

---

## 📈 Resultados Esperados

- 🚀 Alta performance com controle de schema evolutivo
- ✅ Pipeline resiliente e idempotente
- 🔄 Pronto para ser usado em um **workflow orquestrado** ou **job programado**

---




| **Informações**    |            **Detalhes**         |
|--------------------|---------------------------------|
| Nome da Tabela     |          Camada Bronze          |
| Data da Ingestao   |            31/03/2025           |
| Ultima Atualizaçao |            31/03/2025           |
| Origem             | Banco de Dados Azure - Patrick  |
| Responsável        |           Lucas Sousa           |
| Motivo             | Criação de Camadas Bronze       |
| Observações        |               None              |

##INGESTÃO EM FORMATO CSV - TABELA CLIENTES

In [0]:
# 📦 Pipeline Bronze - Ingestão de Dados Clientes (Arquivo CSV)
# 🧠 Objetivo: Realizar a ingestão eficiente, estruturada e confiável de dados de clientes para a camada Bronze

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import current_timestamp, to_date

# ------------------------------------------
# ✅ Caminhos do pipeline (todos no container 'bronze' para padronização e rastreabilidade)
# ------------------------------------------

path_source = 'abfss://transient@dlsprojetofixo.dfs.core.windows.net/ecommerce_customers_500.csv'  # Fonte CSV
path_target = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/clientes_lucas'                   # Destino Delta Lake
checkpoint_path = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/clientes_lucas_checkpoint'   # Checkpoint streaming
schema_location = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/clientes_lucas_schema'       # Schema do Auto Loader

# ------------------------------------------
# ✅ Definição explícita do schema do CSV
# ------------------------------------------

schema = StructType([
    StructField("CustomerID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Email", StringType(), True),
    StructField("Phone", StringType(), True),
    StructField("Date_Time_Load", TimestampType(), True)  # Será substituída pelo timestamp real de ingestão
])

# ------------------------------------------
# ✅ Criação do banco de dados 'bronze' caso não exista
# ------------------------------------------
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

# ------------------------------------------
# ✅ Leitura com Auto Loader (cloudFiles)
# ------------------------------------------

# Auto Loader permite leitura eficiente e incremental dos arquivos
# - cloudFiles.format define o tipo (CSV)
# - schemaLocation guarda o schema detectado, útil para schema evolution
# - schema define os tipos esperados, evitando inferência automática

raw_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.schemaLocation", schema_location)
        .schema(schema)
        .load(path_source)
)

# ------------------------------------------
# ✅ Transformações nos dados
# ------------------------------------------

# Substitui Date_Time_Load pela data e hora de ingestão do Spark
transformed_df = raw_df.withColumn("Date_Time_Load", current_timestamp())

# ✅ Limpeza básica de dados (essencial na Bronze)
# - Remove duplicatas e registros nulos
transformed_df = transformed_df.dropDuplicates().na.drop()

# Cria coluna de particionamento com apenas a data (sem hora)
transformed_df = transformed_df.withColumn("Data_Particao", to_date("Date_Time_Load"))

# ------------------------------------------
# ✅ Escrita dos dados em Delta Lake com boas práticas
# ------------------------------------------

# - checkpointLocation: permite reinício seguro do stream
# - mergeSchema: aceita mudanças futuras de schema
# - partitionBy: melhora a performance de leitura e filtragem
# - trigger(once=True): executa o pipeline uma única vez (modo batch)

(
    transformed_df.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", checkpoint_path)
        .option("mergeSchema", "true")
        .partitionBy("Data_Particao")
        .trigger(once=True)
        .option("path", path_target)
        .toTable("bronze.clientes")
)

# Remove arquivos não referenciados com retenção de 7 dias
spark.sql("VACUUM bronze.clientes RETAIN 168 HOURS")


# ✅ Log de sucesso no notebook
print("✅ Pipeline de ingestão de clientes concluído com sucesso na camada Bronze!")

✅ Pipeline de ingestão de clientes concluído com sucesso na camada Bronze!


In [0]:
%sql select * from bronze.clientes LIMIT 10

CustomerID,Name,PhoneNumber,Age,Date_Time_Load
1,Peter Arroyo,+1-510-4943,35,2025-04-09T00:16:28.167Z
2,Noah Kramer,+1-322-2452,36,2025-04-09T00:16:28.167Z
3,Kirk Becker,+1-554-6529,69,2025-04-09T00:16:28.167Z
4,Jacqueline Gray,+1-700-4444,58,2025-04-09T00:16:28.167Z
5,Julie Hobbs,+1-157-9744,51,2025-04-09T00:16:28.167Z
6,Philip Thompson,+1-843-5051,47,2025-04-09T00:16:28.167Z
7,Lisa Becker,+1-381-5680,41,2025-04-09T00:16:28.167Z
8,Marie Moon,+1-917-7575,41,2025-04-09T00:16:28.167Z
9,Michael Johnson,+1-400-5224,48,2025-04-09T00:16:28.167Z
10,Victor Williams,+1-188-8635,70,2025-04-09T00:16:28.167Z


## INGESTÃO EM FORMATO JSON - TABELA VENDAS

In [0]:
# 📦 Pipeline Bronze - Ingestão de Dados com Auto Loader (Delta Lake + JSON)
# 🧠 Objetivo: Ingestão eficiente, econômica e resiliente de dados brutos em JSON para camada Bronze

from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# ------------------------------------------
# ✅ Caminhos do pipeline (todos no container 'bronze' para padronização e rastreabilidade)
# ------------------------------------------
path_source = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/ecommerce_large.json'  # Origem dos arquivos JSON
path_target = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas'          # Destino Delta Lake
checkpoint_path = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/_checkpoints/vendas_lucas'  # Estado da ingestão
schema_path = 'abfss://bronze@dlsprojetofixo.dfs.core.windows.net/_schema/vendas_lucas'            # Metadados do schema inferido

# ✅ Definição explícita do schema (evita erro de inferência se pasta estiver vazia)
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("order_date", StringType(), True)
])

# ✅ Leitura com Auto Loader (cloudFiles) e schema definido
# - Monitoramento automático de novos arquivos
# - Schema persistente para evolução segura
# - Tolerante a falhas e ideal para ingestão contínua ou programada
raw_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", schema_path)
        .schema(schema)
        .load(path_source)
)

# ✅ Limpeza básica de dados (essencial na Bronze)
# - Remove duplicatas e registros nulos
clean_df = raw_df.dropDuplicates().na.drop()

# ✅ Enriquecimento com coluna de auditoria
# - Marca o momento da ingestão dos dados
bronze_df = clean_df.withColumn("data_carga", current_timestamp())

# ✅ Criação do banco de dados Bronze (se ainda não existir)
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

# ✅ Escrita em Delta Lake com boas práticas
# - checkpointLocation: permite reinício seguro em caso de falhas
# - mergeSchema: aceita novas colunas automaticamente
# - partitionBy: organiza fisicamente os arquivos por data da carga
# - trigger(once=True): executa o pipeline uma vez e encerra (ideal para batch)
(
    bronze_df.writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint_path)
        .option("mergeSchema", "true")
        .partitionBy("data_carga")
        .trigger(once=True)
        .outputMode("append")
        .option("path", path_target)
        .toTable("bronze.vendas")
)

# Remove arquivos não referenciados com retenção de 7 dias
spark.sql("VACUUM bronze.vendas RETAIN 168 HOURS")


# ✅ Log de sucesso
print("✅ Pipeline executado com sucesso na tabela bronze.vendas!")

✅ Pipeline executado com sucesso na tabela bronze.vendas!


In [0]:

%sql select * from bronze.vendas LIMIT 10

CustomerID,Items,OrderDate,OrderID,ShippingAddress,Status,TotalAmount,Date_Time_Load
123,"List(List(ef50354b-b782-4fcb-ad8d-81471c3ea7a4, 422.11, Keyboard, 1))",2024-02-08,1,"List(Lake Matthewberg, Pakistan, 29350, WA, 629 Mercado Motorway Apt. 766)",Completed,36.79,2025-04-09T00:16:58.962Z
126,"List(List(376bbc8e-ad01-44dc-8012-13c49020a44d, 37.53, Gaming Chair, 2))",2024-07-24,2,"List(West Jeffrey, Brunei Darussalam, 39678, AK, 101 Ronald Roads)",Cancelled,603.69,2025-04-09T00:16:58.962Z
500,"List(List(0f4940b5-3f26-4f33-bce0-404654c64594, 232.66, Router, 3), List(96e26e0e-6142-4339-8366-0acad6ae9f61, 27.71, Smartwatch, 3), List(a44e433a-19a2-4905-9c21-5f2402388dc4, 101.39, External Hard Drive, 2), List(e6ffb3e7-f8b7-4ca5-9e5a-0fd1f77b70b9, 222.25, Monitor, 1))",2024-01-27,3,"List(Lake Mike, Saint Pierre and Miquelon, 17295, ME, 98322 Lisa Road)",Cancelled,306.05,2025-04-09T00:16:58.962Z
215,"List(List(950b6c30-424c-4709-ace9-7b7de4797c45, 303.86, Gaming Chair, 1), List(78e306ee-993e-4442-a8b8-181865dde069, 139.28, Keyboard, 2), List(d8ff9a48-3d81-4747-bab9-862aef188f23, 132.5, Router, 3), List(77eb8f95-3713-4df0-bb17-f7f9d71511f7, 194.67, Laptop, 3), List(6c2d822b-2ab2-4a2d-b038-b06a7426804d, 160.01, Gaming Chair, 3))",2023-01-02,4,"List(Wardborough, Qatar, 30418, CO, 52559 Garrett Roads)",Completed,142.48,2025-04-09T00:16:58.962Z
85,"List(List(cbdab18c-86b4-4aad-b276-72994d0d808c, 153.66, Scanner, 1), List(f5f2df88-3558-4045-a6df-e097470edd32, 69.01, Smartwatch, 1))",2024-06-07,5,"List(Scottfort, Botswana, 06962, OK, 6294 Cox Forges Suite 579)",Pending,260.02,2025-04-09T00:16:58.962Z
297,"List(List(6836df29-15e0-46e1-b53d-63d57c75dc5e, 341.36, Power Supply, 3))",2023-11-16,6,"List(North Davidshire, Saint Helena, 59379, SC, 73872 Ortiz Fields Suite 280)",Cancelled,551.39,2025-04-09T00:16:58.962Z
144,"List(List(f5500e10-e547-41ee-b382-29fc426b1d0e, 439.84, Router, 1), List(01ca72d5-ca67-42f1-86a3-ccefa54a85b5, 229.2, Printer, 1), List(3370e320-855e-4131-b2c1-64f435c10708, 255.24, Webcam, 3))",2022-12-31,7,"List(Andersonmouth, Iraq, 58727, VT, 2321 Ronald Cliff)",Pending,549.0,2025-04-09T00:16:58.962Z
223,"List(List(1a9cad3e-1d8a-4cee-9e9e-37aa024ad178, 435.35, Monitor, 1), List(9b70a813-d4ca-42a9-9d69-7433d60af255, 485.35, Smartphone, 1), List(bdb24cec-29bd-4339-8118-6b2ea6d16baa, 11.09, Power Supply, 2), List(ac1adafe-106a-4ebb-b23e-14479cbc7ecc, 293.18, Smartphone, 2))",2023-10-20,8,"List(North Melanie, Norfolk Island, 62596, SC, 188 Reyes Alley)",Completed,533.41,2025-04-09T00:16:58.962Z
420,"List(List(0f6cfadf-bb5b-4ba3-94ae-ac8c0c5396f3, 245.35, Webcam, 1), List(d01e6a0a-7779-4f42-af0d-f91fe1493667, 295.57, Mouse, 2), List(9bb52eb2-2945-4f34-9c60-3616d99041d1, 416.37, Laptop, 3))",2023-05-05,9,"List(West Mathew, Denmark, 04118, NM, 14607 Smith Hills Apt. 583)",Pending,561.69,2025-04-09T00:16:58.962Z
60,"List(List(b8441ad2-cb50-4526-9ca0-58dc3add602d, 401.35, Power Supply, 2), List(b98753e7-3660-48cc-9042-a9f2b09f33a8, 91.16, Charger, 2), List(5c929555-2c72-4ba4-8675-982c934d328d, 172.52, Smartwatch, 3), List(2dfdeab4-2913-465f-aadc-7ddcb0fdab53, 428.85, Keyboard, 1), List(b90663bb-ccc8-42b9-b7a4-f15085e2275f, 107.87, USB Cable, 1))",2024-04-25,10,"List(Smithtown, Seychelles, 39360, MO, 116 Reid Island)",Pending,781.29,2025-04-09T00:16:58.962Z


#FUNÇÃO **EXPLODE** ##
Descompactar valores das colunas do tipo **ARRAY** conforme coluna **Items** e **ShippingAdress**  - Fonte Tabela (bronze.vendas)

A função **Explode()** no Apache Spark é usada para desnormalizar dados estruturados, transformando arrays ou mapas em múltiplas linhas. Isso é útil para lidar com dados aninhados em formatos como JSON ou Parquet

In [0]:
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# ========================================
# 1. CONFIGURAÇÕES INICIAIS E CAMINHOS
# ========================================

# Caminhos no Data Lake (container Bronze no ADLS Gen2)
file_path = "abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas"
path_target = "abfss://bronze@dlsprojetofixo.dfs.core.windows.net/vendas_lucas_exploded"
checkpoint_location = "abfss://bronze@dlsprojetofixo.dfs.core.windows.net/_checkpoint/vendas_lucas_exploded"

# Nome do banco e tabela no catálogo Hive/Unity Catalog
schema_name = "bronze"
table_name = "vendas_lucas_exploded"

# ========================================
# 2. CONFIGURAÇÃO DOS SCHEMAS DE ESTRUTURAS ANINHADAS
# ========================================

# Define o schema para a estrutura aninhada "Items"
items_schema = StructType([
    StructField("ItemID", StringType(), True),
    StructField("Price", DoubleType(), True),
    StructField("ProductName", StringType(), True),
    StructField("Quantity", IntegerType(), True)
])

# Define o schema para "ShippingAddress"
shipping_address_schema = StructType([
    StructField("City", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("PostalCode", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Street", StringType(), True)
])

# ========================================
# 3. LEITURA DOS DADOS COM AUTO LOADER
# ========================================

# Leitura incremental utilizando Auto Loader (cloudFiles)
# Permite escalabilidade, economia de cluster e detecção automática de novos arquivos
# Schema evolution e gerenciamento automático de arquivos

df = spark.readStream.format("delta") \
    .option("cloudFiles.format", "delta") \
    .option("cloudFiles.schemaLocation", "/mnt/schemas/vendas_lucas") \
    .load(file_path)

# ========================================
# 4. TRANSFORMAÇÃO DOS DADOS
# ========================================

# Realiza o explode na coluna de array "Items", transformando múltiplos itens em linhas distintas
# Mantém as colunas principais do pedido e extrai campos internos do item e endereço

df_exploded = df.withColumn("Item", explode(col("Items")))

# Seleciona as colunas finais com renomeação clara para facilitar análise e governança
final_df = df_exploded.select(
    col("OrderDate"),
    col("OrderID"),
    col("CustomerID"),
    col("Status"),
    col("Item.ItemID").alias("ItemID"),
    col("Item.ProductName").alias("ProductName"),
    col("Item.Quantity").alias("Quantity"),
    col("Item.Price").alias("Price"),
    col("TotalAmount"),
    col("ShippingAddress.City").alias("City"),
    col("ShippingAddress.Country").alias("Country"),
    col("ShippingAddress.PostalCode").alias("PostalCode"),
    col("ShippingAddress.State").alias("State"),
    col("ShippingAddress.Street").alias("Street"),
    col("Date_Time_Load")
)

# ========================================
# 5. ESCRITA DO STREAM COM DELTA LAKE
# ========================================

# Escrita do DataFrame em modo append utilizando Delta Lake
# Checkpoint permite reinício seguro sem duplicação
# PartitionBy em OrderDate melhora performance de leitura em queries temporais
# Trigger once executa o stream como batch controlado, com menor custo e uso do cluster

query = final_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_location) \
    .option("mergeSchema", "true") \
    .partitionBy("OrderDate") \
    .trigger(once=True) \
    .start(path_target)

# Aguarda o fim da execução do streaming (modo batch seguro)
query.awaitTermination()

# ========================================
# 6. REGISTRO DA TABELA NO CATÁLOGO (CASO NÃO EXISTA)
# ========================================

# Criação do banco de dados no metastore se ainda não existir
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema_name}")

# Criação manual da tabela baseada nos arquivos já gravados, caso não esteja registrada
if not spark._jsparkSession.catalog().tableExists(f"{schema_name}.{table_name}"):
    spark.sql(f"""
        CREATE TABLE {schema_name}.{table_name}
        USING DELTA
        LOCATION '{path_target}'
    """)

# ========================================
# 7. PÓS-PROCESSAMENTO: CONTAGEM E OTIMIZAÇÃO
# ========================================

# Lê a tabela já gravada (modo batch) para análise posterior
# Isso garante que estamos operando sobre dados estáveis após o stream

df_final = spark.read.format("delta").load(path_target)

# Conta o número total de registros processados após o explode e escrita
# Essa métrica é essencial para auditoria e validação do pipeline

total_linhas = df_final.count()
print(f"✅ Total de linhas após transformação e gravação: {total_linhas}")

# ========================================
# 8. OTIMIZAÇÃO E Z-ORDER
# ========================================

# Aplica técnica de otimização com Z-ORDER sobre colunas críticas
# Essa estratégia reorganiza fisicamente os dados para acelerar consultas com filtros por OrderID e CustomerID
# Ideal para reduzir tempo de leitura e custo de I/O em workloads analíticos complexos

spark.sql(f"""
    OPTIMIZE {schema_name}.{table_name}
    ZORDER BY (OrderID, CustomerID)
""")

print("✅ Pipeline finalizado com sucesso e tabela otimizada!")


✅ Total de linhas após transformação e gravação: 29945
✅ Pipeline finalizado com sucesso e tabela otimizada!


In [0]:
%sql select * from bronze.vendas_lucas_exploded LIMIT 10

OrderDate,OrderID,CustomerID,Status,ItemID,ProductName,Quantity,Price,TotalAmount,City,Country,PostalCode,State,Street,Date_Time_Load
2022-12-03,31,198,Cancelled,b5a7360a-a833-446a-9db2-697e83b335a9,Monitor,1,80.17,276.05,Ericstad,Romania,43407,IN,31942 Sanchez Grove,2025-04-09T00:16:58.962Z
2022-12-03,31,198,Cancelled,c5026804-4c82-4e16-8b47-d6d34ac17e83,Headphones,1,339.26,276.05,Ericstad,Romania,43407,IN,31942 Sanchez Grove,2025-04-09T00:16:58.962Z
2022-12-03,1052,311,Cancelled,732a8215-9e95-4617-8bae-0253584e141b,Scanner,1,345.94,776.54,West Michaelville,Mayotte,97677,CO,2621 Matthew Groves,2025-04-09T00:16:58.962Z
2022-12-03,1052,311,Cancelled,73b85fb9-1cf4-4b7f-b2d8-aebf20037bf1,Router,3,53.11,776.54,West Michaelville,Mayotte,97677,CO,2621 Matthew Groves,2025-04-09T00:16:58.962Z
2022-12-03,1052,311,Cancelled,a6f647a3-d5de-4122-b3cb-fe3da5d1022f,Gaming Chair,2,454.57,776.54,West Michaelville,Mayotte,97677,CO,2621 Matthew Groves,2025-04-09T00:16:58.962Z
2022-12-03,1052,311,Cancelled,5b345cf6-aef8-4341-be1e-465b48aa671f,Printer,2,470.79,776.54,West Michaelville,Mayotte,97677,CO,2621 Matthew Groves,2025-04-09T00:16:58.962Z
2022-12-03,1118,272,Completed,fcb2ec93-3fa5-4256-8300-9287597b81d3,Printer,2,433.37,215.35,Hendersonview,Syrian Arab Republic,99399,PA,53044 Sarah Spur,2025-04-09T00:16:58.962Z
2022-12-03,1118,272,Completed,3f42205e-5fc4-4c4a-b670-0f101a25e9a8,USB Cable,3,19.16,215.35,Hendersonview,Syrian Arab Republic,99399,PA,53044 Sarah Spur,2025-04-09T00:16:58.962Z
2022-12-03,1205,239,Pending,e9c8de14-8948-4635-a1dd-ce1bcaa33988,Tablet,3,171.27,717.73,North Nicholasbury,Sudan,52897,KS,54207 Warner Villages Apt. 371,2025-04-09T00:16:58.962Z
2022-12-03,1205,239,Pending,0e61542b-f7cb-4763-bc4d-862b990aa5f4,Scanner,2,81.4,717.73,North Nicholasbury,Sudan,52897,KS,54207 Warner Villages Apt. 371,2025-04-09T00:16:58.962Z
